# Imports

## PyGAD

In [1]:
import pygad
import numpy as np

import os
import sys
import Bio
import pandas as pd

## spliceAI

In [2]:
from metrics import splicing_gpu
import general_functions
from multiprocessing import Pool

Using TensorFlow backend.


# Multi-threa

In [3]:
from multiprocessing import Pool
import subprocess

# CAI

In [4]:
import time
import hashlib


# BAI

In [5]:
my_env = os.environ.copy()
my_env["PATH"] = "/data/software/anaconda3/envs/bfx/lib/perl5/site_perl/5.22.0:/data/software/anaconda3/pkgs/texlive-selected-20160715-1/share/texlive/tlpkg:" + my_env["PATH"]


In [6]:
# def readFasta( fastaFile: str ):
#     fastaId = ''
#     fastaSeq = ''
#     if os.path.exists( fastaFile ):
#         try:
#             with open( fastaFile, "r") as handle:
#                 cont = 0
#                 fastaIds = []
#                 fastaSeqs = []
#                 for record in SeqIO.parse( handle, "fasta"):
#                     fastaIds.append(record.id)
#                     fastaSeqs.append(record.seq)
#         except IOError as e:
#             print( "I/O error({0}): {1}".format(e.errno, e.strerror) )
#         except:
#             print( "Unexpected error:", sys.exc_info()[0] )
#     else:
#         print("File does not exist:" + fastaFile )
#         exit(2)
#     return (fastaIds, fastaSeqs)


# def predictSites( fastaSeq: str ):
#     #config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=100,
#     #                        inter_op_parallelism_threads=100,
#     #                        allow_soft_placement=True,
#     #                        device_count = {'CPU': 100})
#     #session = tf.compat.v1.Session(config=config)
#     #K.set_session(session)
    
#     from keras.models import load_model
#     import tensorflow as tf
#     from tensorflow import keras

#     gpus =tf.config.experimental.list_physical_devices('GPU')
#     tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1500)])

#     context = 10000
#     keras.backend.set_learning_phase(0)
#     i=0
#     paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
#     models = [load_model(resource_filename('spliceai', x), compile=False ) for x in paths]
#     x = one_hot_encode('N'*(context//2) + fastaSeq + 'N'*(context//2))[None, :]
#     y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
#     acceptor_prob = y[0, :, 1]
#     donor_prob = y[0, :, 2]
#     coord = list( range(1, len( donor_prob ) ) )
#     return ( coord, acceptor_prob, donor_prob )


# def savePredictions( predictions: list, outfile: str ):
#     original_stdout = sys.stdout # Save a reference to the original standard output
#     with open( outfile, "w") as handle:
#         sys.stdout = handle # Change the standard output to the file we created.
#         print("coord","acceptor_prob","donor_prob", sep="\t")
#         for row_ind in range(0,len( predictions[0]) - 1):
#             for col in predictions:
#                 print( col[row_ind], end="\t" )
#             print("")
#     sys.stdout = original_stdout # Reset the standard output to its original value

# Start

## Parameters

### Metrics

In [7]:
cai_on = True
bai_on = True
sps_on = True
cpg_on = True
pas_on = False


### Weights

In [8]:
cai_w = 1
bai_w = 1
sps_w = 1
cpg_w = 1
pas_w = 1

### Locations

In [9]:
ga_input = '/grid/home/nbourgeois/data/test_sequences/hidua/hidua.pep.fas'
# ga_input = '/grid/home/nbourgeois/data/test_sequences/eGFP/pKITR-LP1-eGFP-004_protein_seq.fa'
cai_index_loc = 'CoCoPUTs_codon_usage/codon_usage/Brain_Cerebellum.codon.txt'
bai_index_loc = 'CoCoPUTs_codon_usage/bicodon_usage/Brain_Cerebellum.bicodon.txt'


codon_usage_table_loc = 'codon_usage.getex.txt'

# Algorithm

### Read in File

In [10]:
(keys, seqs) = general_functions.readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

### Create Codon translation table

In [11]:
codon_usage_table = pd.read_csv(codon_usage_table_loc,sep='\t')
forward_table = pd.Series(codon_usage_table.AA.values,index=codon_usage_table.Codon).to_dict()

back_table = {}
for key in forward_table:
    val = forward_table[key]
    if val not in back_table.keys():
        back_table[val] = [key]
    else:
        back_table[val].append(key)

gene_space = []
for aa in aa_seq:
    all_cds = back_table[aa]
    gene_space.append(all_cds)

codon_to_int = {}
i=0
for codon in forward_table.keys():
    codon_to_int[codon] = i
    codon_to_int[i] = codon
    i += 1
    
gene_space_int = [[codon_to_int[x] for x in y] for y in gene_space]

# Fitness Function

In [12]:
def get_sps(seq_aa):
    
    prediction_ind, acceptor_prob, donor_prob = splicing_gpu.predictSites( seq_aa )
    if any(acceptor_prob > .8) or any(donor_prob > .8):
        val = 1 
    else: 
        val = 0

    return(val)

def get_bai(seq_aa):
    
    bai_script = ' '.join(['nice','perl','bai.pl',seq_aa,bai_index_loc])
    out = subprocess.run(bai_script, shell=True, capture_output=True, env=my_env).stdout
    bai = float(out)
    
    return(bai)

def get_cai(seq_aa):
    seq_aa_bin = seq_aa.encode('utf-8')
    hash_object = hashlib.sha1(seq_aa_bin)
    hex_dig = hash_object.hexdigest()
    idx = hex_dig[0:6]
    
    if not os.path.isdir('tmp'):
        os.mkdir('tmp') 
    file_fa = os.path.join('tmp', str(idx)+'.fa')
    file_ca = os.path.join('tmp', str(idx)+'.ca')
    
    with open(file_fa ,'w') as fileo:
        fileo.write('>fa\n'+seq_aa+'\n')
        
    cmd = 'nice cai ' + file_fa + ' -cfile ' + cai_index_loc + ' -outfile '+ file_ca
    subprocess.Popen(cmd, shell=True)

    # wait for file to refresh
    i=0
    while not os.path.isfile(file_ca):
        if i > 100:
            print('File not generated:'+file_ca)
            return(0)
        time.sleep(.1)
        i+=1
    
    with open(file_ca,'r') as fileo:
        lines = fileo.readline()
    
    cai = float(lines.split()[-1])
        
#     cmd = 'rm '+ file_ca +' '+ file_ca
#     subprocess.Popen(cmd, shell=True)
    
    return(cai)


def get_cpg(seq):
    count = seq.count("CG")
    cpg_score = ( len(seq) - count * 2 ) / len(seq)  
    return(cpg_score)

def get_pas(seq_aa):
    seq_aa_bin = seq_aa.encode('utf-8')
    hash_object = hashlib.sha1(seq_aa_bin)
    hex_dig = hash_object.hexdigest()
    idx = hex_dig[0:6]
    
    if not os.path.isdir('tmp'):
        os.mkdir('tmp') 
    file_fa = os.path.join('tmp', str(idx)+'.fa')
    file_prefix = os.path.join('tmp', str(idx)+'_pasta')
    file_out = file_prefix+'.pas.txt'
    
    if not os.path.isfile(file_fa):
        with open(file_fa ,'w') as fileo:
            fileo.write('>fa\n'+seq_aa+'\n')
        
    cmd = 'nice sh deep_pasta.sh ' + file_fa + ' ' + file_prefix
    subprocess.Popen(cmd, shell=True)

    # wait for file to refresh
    i=0
    while not os.path.isfile(file_out):
        if i > 3000:
            print('File not generated:'+file_out)
            return(0)
        time.sleep(.1)
        i+=1
    
    with open(file_out,'r') as fileo:
        lines = fileo.readline()
    
#     print(lines)
    if len(lines.split()) < 2:
        return(0)
    
    pas = float(lines.split()[-1])
            
    return(pas)

def get_memory():
    with open('/proc/meminfo', 'r') as mem:
        free_memory = 0
        for i in mem:
            sline = i.split()
            if str(sline[0]) in ('MemFree:', 'Buffers:', 'Cached:'):
                free_memory += int(sline[1])
    return free_memory


def fitness_func(solution, solution_idx):
    
    if not type(solution) is str:
        seq_aa = ''.join([codon_to_int[x] for x in solution])
    else:
        seq_aa = solution
#     print(solution_idx)

    tmp_dict = {}
    
    #Check for redundancy
    if seq_aa in all_sols.keys():
        fitness = all_sols[seq_aa]['fitness']

    else:
        fitness = 0
        total_weight=0
        
        if cai_on:
            cai = get_cai(seq_aa)
            fitness += cai*cai_w
            total_weight += cai_w
            tmp_dict['cai'] = cai
        
        if bai_on:
            bai = get_bai(seq_aa)
            fitness += bai*bai_w
            total_weight += bai_w
            tmp_dict['bai'] = bai
            
        if cpg_on:
            cpg = get_cpg(seq_aa)
            fitness += cpg*cpg_w
            total_weight += cpg_w
            tmp_dict['cpg'] = cpg

        if sps_on:
            sps = get_sps(seq_aa)
            fitness += sps*sps_w
            total_weight += sps_w
            tmp_dict['sps'] = sps

        if pas_on:
            pas = get_pas(seq_aa)
            fitness += pas*pas_w
            total_weight += pas_w
            tmp_dict['pas'] = pas

        fitness = fitness/total_weight
        tmp_dict['fitness'] = fitness
        all_sols[seq_aa] = tmp_dict
        
        #     output = numpy.sum(solution*function_inputs)
        #     fitness = 1.0 / numpy.abs(output - desired_output)
    
    return fitness



# Genetic Algorithm

In [13]:
"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

# def convert_aa_int():
    
desired_output = 1 # Function output.
all_sols = {}

fitness_function = fitness_func

num_generations = 100 # Number of generations.
num_parents_mating = 5 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 20 # Number of solutions in the population.
num_genes = len(gene_space)

# init_range_low = -2
# init_range_high = 5

parent_selection_type = "sss" # Type of parent selection.
keep_parents = 5 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 5 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0

def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.

def fitness_wrapper(solution):
    return fitness_func(solution, 0)


class PooledGA(pygad.GA):

    def cal_pop_fitness(self):
        global pool

        pop_fitness = pool.map(fitness_wrapper, self.population)
        pop_fitness = np.array(pop_fitness)
        return pop_fitness

ga_instance_pool = PooledGA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
#                        init_range_low=init_range_low,
#                        init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=callback_generation,
                       gene_type=int,
                       gene_space=gene_space_int,
)

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
#                        init_range_low=init_range_low,
#                        init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=callback_generation,
                       gene_type=int,
                       gene_space=gene_space_int,
)


In [14]:
# %%time

# # Running the GA to optimize the parameters of the function.
# ga_instance.run()

# # After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
# ga_instance.plot_result()

# # Returning the details of the best solution.
# solution, solution_fitness, solution_idx = ga_instance.best_solution()
# print("Parameters of the best solution : {solution}".format(solution=solution))
# seq_aa = ''.join([codon_to_int[x] for x in solution])
# print("NT Sequence of the best solution : {seq_aa}".format(seq_aa=seq_aa))
# print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
# print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

# # # prediction = numpy.sum(numpy.array(function_inputs)*solution) Replace
# # print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

# if ga_instance.best_solution_generation != -1:
#     print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# # Saving the GA instance.
# filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
# ga_instance.save(filename=filename)

# # Loading the saved GA instance.
# loaded_ga_instance = pygad.load(filename=filename)
# loaded_ga_instance.plot_result()

In [ ]:
%%time

print('Started')
with Pool(processes=5) as pool:

    # Running the GA to optimize the parameters of the funcabstion.
    ga_instance_pool.run()

    # After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
    ga_instance_pool.plot_result()

    # Returning the details of the best solution.
    solution, solution_fitness, solution_idx = ga_instance_pool.best_solution()
    print("Parameters of the best solution : {solution}".format(solution=solution))
    print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
    print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

    solution_aa = ''.join([codon_to_int[x] for x in solution])
   
    print("Predicted output based on the best solution : {prediction}".format(prediction=solution_aa))

#     if ga_instance_pool.best_solution_generation != -1:
#         print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

    # Saving the GA instance.
    filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
    ga_instance_pool.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_result()

Started


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1321, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 5, in <module>
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/pygad/pygad.py", line 1170, in run
    self.last_generation_fitness = self.cal_pop_fitness()
  File "<ipython-input-13-a7cdf53c9b30>", line 56, in cal_pop_fitness
    pop_fitness = pool.map(fitness_wrapper, self.population)
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/multiprocessing/pool.py", line 268, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/multiprocessing/pool.py", line 651, in get
    self.wait(timeout)
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/multiprocessing/pool.py", line 648, in wait
    self._event.wait(timeout)
  File "/data/software/anaconda3/

In [ ]:
solution_aa
# seq_aa = ''.join([codon_to_int[x] for x in solution])
# all_sols[seq_aa]
Pool(processes=5)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-d0b568b60e42>", line 1, in <module>
    solution_aa
NameError: name 'solution_aa' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offs